<a href="https://colab.research.google.com/github/sapperka/AZSII/blob/main/%D0%90%D0%97%D0%A1%D0%98%D0%98_%D0%91%D0%91%D0%9C%D0%9E_02_23_%D0%A0%D1%8B%D0%B1%D0%B0%D0%BA%D0%BE%D0%B2_%D0%9F%D0%A08.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# АЗСИИ

## Практика 8: Методы защиты от атак на модели ИИ

Группа: ББМО-02-23

Студент: Рыбаков Владислав Андреевич

## Цель задания:
Изучить методы защиты моделей ИИ от различных атак, включая методы защиты на уровне данных,
моделирования и обучения. Реализовать эти методы и проверить их эффективность против атак,
изученных ранее.

## Задачи:
1. Изучить и реализовать защиту модели с помощью тренировок на противоречивых примерах
(Adversarial Training).
2. Реализовать метод защиты на основе градиентной маскировки.
3. Использовать регуляризацию и нормализацию для повышения устойчивости модели.
4. Проверить эффективность методов защиты против атак FGSM, PGD и GAN-based атак.
5. Оценить улучшение точности защищенной модели на противоречивых примерах.

## Шаги выполнения:

### Шаг 1: Защита с помощью Adversarial Training
Adversarial Training — это метод защиты, который заключается в том, чтобы обучать модель на
противоречивых примерах. Этот метод помогает модели научиться быть более устойчивой к атакам,
так как она сталкивается с противоречивыми примерами на этапе обучения.

In [11]:
import tensorflow as tf
import numpy as np
from tensorflow.keras.datasets import mnist

# Загрузка данных MNIST
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

# Нормализация данных
train_images = train_images / 255.0
test_images = test_images / 255.0

# Преобразование меток в one-hot encoding
train_labels = tf.keras.utils.to_categorical(train_labels, 10)
test_labels = tf.keras.utils.to_categorical(test_labels, 10)

# Уменьшаем количество обучающих изображений до 1000:
train_images = train_images[:1000]
train_labels = train_labels[:1000]

# Функция FGSM атаки
def fgsm_attack(image, epsilon, gradient):
  perturbation = epsilon * np.sign(gradient)
  adversarial_image = image + perturbation
  adversarial_image = np.clip(adversarial_image, 0, 1)
  return adversarial_image

# Функция для генерации противоречивых примеров:
def generate_adversarial_examples(model, images, labels, epsilon):
    adversarial_images = []
    for i in range(len(images)):
        image = tf.convert_to_tensor(images[i].reshape((1, 28, 28, 1)), dtype=tf.float32)
        label = tf.convert_to_tensor(labels[i].reshape((1, 10)), dtype=tf.float32)

        with tf.GradientTape() as tape:
            tape.watch(image)
            prediction = model(image)
            loss = tf.keras.losses.categorical_crossentropy(label, prediction)
        gradient = tape.gradient(loss, image)
        adversarial_image = fgsm_attack(image.numpy(), epsilon, gradient.numpy())
        adversarial_images.append(adversarial_image.reshape(28, 28))

    return np.array(adversarial_images)

# Создание модели
def create_model():
  model = tf.keras.Sequential([
  tf.keras.layers.Flatten(input_shape=(28, 28)),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dense(10, activation='softmax')
  ])
  model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
  return model

# Обучение модели с противоречивыми примерами
def adversarial_training(model, train_images, train_labels, epsilon):
  for epoch in range(5): # Короткое обучение для демонстрации
    for i in range(0, len(train_images), 32):
      batch_images = train_images[i:i+32]
      batch_labels = train_labels[i:i+32]

      # Генерация противоречивых примеров для текущей партии данных
      adversarial_images = generate_adversarial_examples(model, batch_images, batch_labels, epsilon)

      # Объединение оригинальных и противоречивых примеров
      combined_images = np.concatenate([batch_images, adversarial_images], axis=0)
      combined_labels = np.concatenate([batch_labels, batch_labels], axis=0)

      # Обучение на комбинированных данных
      model.train_on_batch(combined_images, combined_labels)

# Инициализация модели
model = create_model()

# Тренировка модели с защитой на противоречивых примерах
adversarial_training(model, train_images, train_labels, epsilon=0.1)

# Сохранение защищенной модели
model.save('adversarial_trained_model.h5')

### Шаг 2: Градиентная маскировка (Gradient Masking)
Gradient Masking — это метод защиты, который затрудняет доступ к градиентам модели для атак. Он
используется для уменьшения информации, доступной для атакующих, и усложнения поиска
направленных изменений.

In [12]:
# Реализация градиентной маскировки
# Для демонстрации мы можем использовать специальные функции активации
import tensorflow as tf
from tensorflow.keras.layers import Activation
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical

# Обновление модели для градиентной маскировки
def create_masked_model():
  model = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape=(28, 28)),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(10),
    Activation('softplus') # Используем softplus вместо softmax для градиентной маскировки
  ])
  model.compile(optimizer='adam', loss='categorical_crossentropy',
  metrics=['accuracy'])
  return model

# Обучение модели с градиентной маскировкой
masked_model = create_masked_model()
masked_model.fit(train_images, train_labels, epochs=5)

# Сохранение модели с градиентной маскировкой
masked_model.save('masked_model.h5')

Epoch 1/5
32/32 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.3701 - loss: 1.9729
Epoch 2/5
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.8202 - loss: 0.8462
Epoch 3/5
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.8783 - loss: 0.4953
Epoch 4/5
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.9203 - loss: 0.3440
Epoch 5/5
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.9425 - loss: 0.2664


### Шаг 3: Регуляризация и нормализация для повышения устойчивости
Использование таких методов, как L2-регуляризация, дропаут и нормализация батчей, может помочь
улучшить устойчивость модели к атакам.

In [13]:
# Модель с регуляризацией и нормализацией
def create_regularized_model():
  model = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape=(28, 28)),
    tf.keras.layers.Dense(128, activation='relu',
    kernel_regularizer=tf.keras.regularizers.l2(0.01)),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dense(10, activation='softmax')
  ])
  model.compile(optimizer='adam', loss='categorical_crossentropy',
  metrics=['accuracy'])
  return model

# Обучение модели с регуляризацией и нормализацией
regularized_model = create_regularized_model()
regularized_model.fit(train_images, train_labels, epochs=5)

# Сохранение модели с регуляризацией
regularized_model.save('regularized_model.h5')

Epoch 1/5
32/32 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - accuracy: 0.2498 - loss: 4.2306
Epoch 2/5
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7030 - loss: 2.2155
Epoch 3/5
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.8007 - loss: 1.7290
Epoch 4/5
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.8647 - loss: 1.4396
Epoch 5/5
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.8554 - loss: 1.3311


### Шаг 4: Оценка моделей на противоречивых примерах
Теперь проверим эффективность всех защитных методов на атакованных данных, созданных с
помощью FGSM и других методов, таких как PGD или GAN.

In [14]:
# Загрузка моделей:
models = {
    'Protected_Model': tf.keras.models.load_model('adversarial_trained_model.h5'),
    'Regularized_Model': tf.keras.models.load_model('regularized_model.h5'),
    'Masked_Model': tf.keras.models.load_model('masked_model.h5')
}

# Генерация противоречивых примеров
adversarial_test_images = generate_adversarial_examples(models['Protected_Model'], test_images, test_labels, epsilon=0.1)

# Оценка моделей на противоречивых примерах
for model_name, model in models.items():
    test_loss, test_acc = model.evaluate(adversarial_test_images, test_labels)
    print(f'Accuracy of {model_name} model on adversarial examples: {test_acc}')

313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.5089 - loss: 1.4444
Accuracy of Protected_Model model on adversarial examples: 0.5641999840736389
313/313 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - accuracy: 0.5591 - loss: 2.2275
Accuracy of Regularized_Model model on adversarial examples: 0.6050999760627747
313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.4385 - loss: 1.7130
Accuracy of Masked_Model model on adversarial examples: 0.487199991941452


### Вывод:

В ходе исследования протестировали различные подходы к защите моделей от атак на основе противоречивых примеров. Для генерации атак использовалась методика FGSM, а в качестве защитных мер применялись градиентная маскировка и метод, основанный на регуляризации и нормализации. Обучение проводилось на данных, включавших как оригинальные изображения, так и сгенерированные противоречивые примеры.

Результаты тестирования показали следующую точность на противоречивых примерах:
1. Protected_Model: точность 56.42% (accuracy: 0.5642).
2. Regularized_Model: точность 60.51% (accuracy: 0.6051).
3. Masked_Model: точность 48.72% (accuracy: 0.4872).

Метод регуляризации оказался наиболее эффективным, продемонстрировав наилучшую точность.